# Predict insurance premium based on age, gender, smoker,...
* Comprehensive EDA
* Fit Random Forest and XGBoost model
* SHAP explanations for XGBoost model
* Blending
* Analyze shortcomings of the model predictions

In [ ]:
# packages
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import time

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import shap

import xgboost as xgb

# Import and first glance

In [ ]:
df = pd.read_csv('../input/insurance-charges/insurance.csv')

In [ ]:
df.head()

In [ ]:
df.shape

# EDA

### Target

In [ ]:
# target
df.charges.plot(kind='hist', bins=20)
plt.title('Target = charges')
plt.grid()
plt.show()

In [ ]:
# target is skewed => add log transformed target
df['log_charges'] = np.log10(df.charges)

In [ ]:
# plot log target
df.log_charges.plot(kind='hist', bins=20)
plt.title('log10(Target)')
plt.grid()
plt.show()

### Numeric features

In [ ]:
df.age.plot(kind='hist', bins=20)
plt.title('Age')
plt.grid()
plt.show()

In [ ]:
df.bmi.plot(kind='hist', bins=20)
plt.title('BMI')
plt.grid()
plt.show()

### Categorical features

In [ ]:
df.sex.value_counts()

In [ ]:
df.sex.value_counts().plot(kind='bar')
plt.title('Sex')
plt.grid()
plt.show()

In [ ]:
df.children.value_counts()

In [ ]:
df.children.value_counts().plot(kind='bar')
plt.title('Children')
plt.grid()
plt.show()

In [ ]:
# add also numeric version
df['children_num'] = pd.to_numeric(df.children)
df.children_num.describe()

In [ ]:
df.smoker.value_counts()

In [ ]:
df.smoker.value_counts().plot(kind='bar')
plt.title('Smoker')
plt.grid()
plt.show()

In [ ]:
df.region.value_counts()

In [ ]:
df.region.value_counts().plot(kind='bar')
plt.title('Region')
plt.grid()
plt.show()

### Correlations

In [ ]:
df[['age', 'bmi','children_num']].corr()

In [ ]:
sns.pairplot(df[['age', 'bmi','children_num']], 
             kind='reg', 
             plot_kws={'line_kws':{'color':'magenta'}, 'scatter_kws': {'alpha': 0.1}})
plt.show()

# Target vs Feature

In [ ]:
df.columns

In [ ]:
# define categorical and numeric features
features_cat = ['sex', 'children', 'smoker', 'region']
features_num = ['age', 'bmi']

In [ ]:
for f in features_cat:
    plt.figure(figsize=(6,4))
    sns.violinplot(x=f, y='log_charges', data=df)
    my_title = 'Target vs ' + f
    plt.title(my_title)
    plt.grid()

In [ ]:
# scatter plot
for f in features_num:    
    plt.scatter(df[f], df.log_charges, alpha=0.25)
    my_title = 'Target vs ' + f
    plt.title(my_title)
    plt.xlabel(f)
    plt.ylabel('log_charges')
    plt.grid()
    plt.show()

In [ ]:
# smoothed visualization
for f in features_num:
    my_title = 'Target vs ' + f
    sns.jointplot(data=df, x=f, y='log_charges', kind='kde')
    plt.title(my_title)
    plt.show()

In [ ]:
# The target vs age plot looks nice, there seem to be separate regions.
# Let's have a closer look:
fig = px.scatter(df, x='age', y='log_charges',
                    color='smoker',
                    opacity=0.5)
fig.update_layout(title='Target vs Age / Smoker')
fig.show()

In [ ]:
# same for BMI instead of Age
fig = px.scatter(df, x='bmi', y='log_charges',
                    color='smoker',
                    opacity=0.5)
fig.update_layout(title='Target vs BMI / Smoker')
fig.show()

In [ ]:
# interactive 3D plot
fig = px.scatter_3d(df, x='age', y='bmi', z='log_charges',
                    color='smoker',
                    opacity=0.5)
fig.update_layout(title='Target vs Age and BMI (smoker via color)')
fig.show()


# Modeling

In [ ]:
# select features
features = features_num + features_cat
features

In [ ]:
X = df[features]
y = df.log_charges # let's use the log transformed target!

In [ ]:
# dummy encoding
X = pd.get_dummies(X)

In [ ]:
X.head()

In [ ]:
# train / test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=180)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

# > Random Forest model

In [ ]:
# define random forest model (regression)
regRF = RandomForestRegressor(n_estimators=500, max_features=6, random_state=1234)

In [ ]:
# fit model
t1 = time.time()
regRF.fit(X_train, y_train)
t2 = time.time()
print('Elapsed time [sec]:', round(t2-t1,3))

In [ ]:
# feature importance
feature_imp = pd.Series(regRF.feature_importances_, index=X.columns).sort_values(ascending=False)
feature_imp

In [ ]:
# and plot
sns.barplot(x=feature_imp, y=feature_imp.index)
plt.title('Variable Importance - RF')
plt.grid()
plt.show()

### Evaluate performance on test set

In [ ]:
# predict values on test set
yhat_test = regRF.predict(X_test)

In [ ]:
# metrics (on log transformed data)
print('RMSE Test (log trafo) :', round(np.sqrt(metrics.mean_squared_error(yhat_test, y_test)),6))
print('MAE Test  (log trafo) :', round(metrics.mean_absolute_error(yhat_test, y_test),6))
print('R^2       (log trafo) :', round(metrics.r2_score(yhat_test, y_test),6))

In [ ]:
plt.scatter(y_test, yhat_test, alpha=0.5)
plt.title('RF Prediction vs Actual [Test] - log transformed data')
plt.grid()
plt.show()

In [ ]:
# transform back to original "coordinates"
yhat_test_orig = 10**yhat_test
y_test_orig = 10**y_test

In [ ]:
plt.scatter(y_test_orig, yhat_test_orig, alpha=0.5)
plt.title('RF Prediction vs Actual [Test] - Original Coordinates')
plt.grid()
plt.show()

In [ ]:
# metrics in original coordinates
rmse_test_RF = round(np.sqrt(metrics.mean_squared_error(yhat_test_orig, y_test_orig)),6)
mae_test_RF = round(metrics.mean_absolute_error(yhat_test_orig, y_test_orig),6)
R2_test_RF = round(metrics.r2_score(yhat_test_orig, y_test_orig),6)
print('RMSE Test :', rmse_test_RF)
print('MAE Test  :', mae_test_RF)
print('R^2       :', R2_test_RF)

In [ ]:
# check calibration
print('Mean actual [test] =        ', y_test_orig.mean())
print('Mean prediction RF [test] = ', yhat_test_orig.mean())
print('Ratio pred/act [test] =     ', yhat_test_orig.mean()/y_test_orig.mean())

In [ ]:
# check calibration on training
yhat_train = regRF.predict(X_train)
yhat_train_orig = 10**yhat_train
y_train_orig = 10**y_train

print('Mean actual [train] =        ', y_train_orig.mean())
print('Mean prediction RF [train] = ', yhat_train_orig.mean())
print('Ratio pred/act [train] =     ', yhat_train_orig.mean()/y_train_orig.mean())

In [ ]:
# scatter plot on training data
plt.scatter(y_train_orig, yhat_train_orig, alpha=0.5)
plt.title('RF Prediction vs Actual [Train] - Original Coordinates')
plt.grid()
plt.show()

# > XGBoost model

In [ ]:
regXGB = xgb.XGBRegressor(             
                    scale_pos_weight=1,
                    learning_rate=0.05,
                    colsample_bytree=0.7,
                    subsample=1,
                    n_estimators=100,
                    reg_alpha=0,
                    max_depth=4,
                    gamma=0,
                    random_state=1234)

In [ ]:
n_estimators = [50,100,200,300,400,500]
max_depth = [4,6]
min_child_weight = [1,5,10]
colsample_bytree = [0.5,0.7]

parameter_grid = {'n_estimators': n_estimators,
                  'max_depth': max_depth,
                  'min_child_weight': min_child_weight,
                  'colsample_bytree': colsample_bytree}

print(parameter_grid)

In [ ]:
# grid search
grid_cv = GridSearchCV(estimator = regXGB,
                       param_grid = parameter_grid,
                       cv=5,
                       n_jobs=-1,
                       verbose=1)

t1 = time.time()
grid_cv.fit(X_train,y_train)
t2 = time.time()

In [ ]:
print('Elapsed time [sec]:', round(t2-t1,3))

In [ ]:
# best parameters from grid search
grid_cv.best_params_

In [ ]:
regXGB = grid_cv.best_estimator_

In [ ]:
feature_impX = pd.Series(regXGB.feature_importances_, index=X.columns).sort_values(ascending=False)
feature_impX

In [ ]:
# and plot
sns.barplot(x=feature_impX, y=feature_impX.index)
plt.title('Variable Importance - XGB')
plt.grid()
plt.show()

In [ ]:
# predict values on test set
yhat_testX = regXGB.predict(X_test)

In [ ]:
# metrics (on log transformed data)
print('RMSE Test (log trafo) :', round(np.sqrt(metrics.mean_squared_error(yhat_testX, y_test)),6))
print('MAE Test  (log trafo) :', round(metrics.mean_absolute_error(yhat_testX, y_test),6))
print('R^2       (log trafo) :', round(metrics.r2_score(yhat_testX, y_test),6))

In [ ]:
plt.scatter(y_test, yhat_testX, alpha=0.5)
plt.title('XGB Prediction vs Actual [Test] - log transformed data')
plt.grid()
plt.show()

In [ ]:
# transform back to original "coordinates"
yhat_testX_orig = 10**yhat_testX

In [ ]:
plt.scatter(y_test_orig, yhat_testX_orig, alpha=0.5)
plt.title('XGB Prediction vs Actual [Test] - Original Coordinates')
plt.grid()
plt.show()

In [ ]:
rmse_test_XGB = round(np.sqrt(metrics.mean_squared_error(yhat_testX_orig, y_test_orig)),6)
mae_test_XGB = round(metrics.mean_absolute_error(yhat_testX_orig, y_test_orig),6)
R2_test_XGB = round(metrics.r2_score(yhat_testX_orig, y_test_orig),6)
print('RMSE Test :', rmse_test_XGB)
print('MAE Test  :', mae_test_XGB)
print('R^2       :', R2_test_XGB)

In [ ]:
# check calibration
print('Mean actual [test] =         ', y_test_orig.mean())
print('Mean prediction XGB [test] = ', yhat_testX_orig.mean())
print('Ratio pred/act [test] =      ', yhat_testX_orig.mean()/y_test_orig.mean())

In [ ]:
# check calibration on training
yhat_trainX = regXGB.predict(X_train)
yhat_trainX_orig = 10**yhat_trainX
y_train_orig = 10**y_train

print('Mean actual [train] =         ', y_train_orig.mean())
print('Mean prediction XGB [train] = ', yhat_trainX_orig.mean())
print('Ratio pred/act [train] =      ', yhat_trainX_orig.mean()/y_train_orig.mean())

In [ ]:
# scatter plot on training data
plt.scatter(y_train_orig, yhat_trainX_orig, alpha=0.5)
plt.title('XGB Prediction vs Actual [Train] - Original Coordinates')
plt.grid()
plt.show()

# > SHAP explanations for XGBoost model

In [ ]:
# init java script visualization
shap.initjs()
# create explainer object
explainer = shap.TreeExplainer(regXGB) 
# calc shap values
shap_values = explainer.shap_values(X)

In [ ]:
# explain an individual observation
my_observation_index = 1
print(X.iloc[my_observation_index])
shap.force_plot(explainer.expected_value, shap_values[my_observation_index,:], X.iloc[my_observation_index,:])

In [ ]:
# show all individual effects in one plot
shap.summary_plot(shap_values, X)

In [ ]:
# calc "feature importance" using SHAP values
shap.summary_plot(shap_values, X, plot_type="bar")

In [ ]:
# SHAP dependece plot
shap.dependence_plot("age", shap_values, X)

#### This plots shows the impact of different values of age on the prediction. It furthermore visualizes also the variable with the strongest interaction with age, in this case smoker. One can see that there is actually an interaction effect between age and smoker as the blue and the red curves are behaving differently.

# > Blend models

In [ ]:
weight_RF = 0.8
weight_XGB = 1-weight_RF
# blend predictions on retransformed values
yhat_blend_test_orig = weight_RF*yhat_test_orig + weight_XGB*yhat_testX_orig

In [ ]:
plt.scatter(y_test_orig, yhat_blend_test_orig, alpha=0.5)
plt.title('Blended Prediction vs Actual - Original Coordinates')
plt.grid()
plt.show()

In [ ]:
rmse_test_blend = round(np.sqrt(metrics.mean_squared_error(yhat_blend_test_orig, y_test_orig)),6)
mae_test_blend = round(metrics.mean_absolute_error(yhat_blend_test_orig, y_test_orig),6)
R2_test_blend = round(metrics.r2_score(yhat_blend_test_orig, y_test_orig),6)
print('RMSE Test :', rmse_test_blend)
print('MAE Test  :', mae_test_blend)
print('R^2       :', R2_test_blend)

In [ ]:
# compare RMSE
print('RMSE Test RF    :', rmse_test_RF)
print('RMSE Test XGB   :', rmse_test_XGB)
print('RMSE Test Blend :', rmse_test_blend)

In [ ]:
# compare MAE
print('MAE Test RF    :', mae_test_RF)
print('MAE Test XGB   :', mae_test_XGB)
print('MAE Test Blend :', mae_test_blend)

In [ ]:
# compare R^2
print('R^2 Test RF    :', R2_test_RF)
print('R^2 Test XGB   :', R2_test_XGB)
print('R^2 Test Blend :', R2_test_blend)

In [ ]:
# check calibration
print('Mean actual [test] =         ', y_test_orig.mean())
print('Mean prediction XGB [test] = ', yhat_blend_test_orig.mean())
print('Ratio pred/act [test] =      ', yhat_blend_test_orig.mean()/y_test_orig.mean())

# Collect and export results

In [ ]:
df_train_export = X_train.copy()
df_train_export['actual'] = y_train_orig
df_train_export['pred_RF'] = yhat_train_orig
df_train_export['pred_XGB'] = yhat_trainX_orig
df_train_export.head()

In [ ]:
df_test_export = X_test.copy()
df_test_export['actual'] = y_test_orig
df_test_export['pred_RF'] = yhat_test_orig
df_test_export['pred_XGB'] = yhat_testX_orig
df_test_export.head()

In [ ]:
# export
df_train_export.to_csv('export_train.csv')
df_test_export.to_csv('export_test.csv')

# Outlier analysis

In [ ]:
# check RF performance for non-smokers vs smokers
train_A = df_train_export[df_train_export.smoker_no==1]
train_B = df_train_export[df_train_export.smoker_no==0]
plt.scatter(train_A.actual, train_A.pred_RF, alpha=0.5, c='blue')
plt.scatter(train_B.actual, train_B.pred_RF, alpha=0.5, c='red')
plt.grid()
plt.show()

#### Interesting: The RF model seems to have an issue with the non-smokers...? What about the XGB model?

In [ ]:
# check XGB performance for non-smokers vs smokers
train_A = df_train_export[df_train_export.smoker_no==1]
train_B = df_train_export[df_train_export.smoker_no==0]
plt.scatter(train_A.actual, train_A.pred_XGB, alpha=0.5, c='blue')
plt.scatter(train_B.actual, train_B.pred_XGB, alpha=0.5, c='red')
plt.grid()
plt.show()

#### Similar situation...